In [2]:
import ee
import geemap
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()

In [12]:
# Let's see how to take a list of numbers and add 1 to each element
myList = ee.List.sequence(1, 10)

# Define a function that takes a number and adds 1 to it
def myFunction(number):
  return number + 1

print(myFunction(1))

#Re-Define a function using Earth Engine API
def myFunction(number):
  return ee.Number(number).add(1)

# Map the function of the list
newList = myList.map(myFunction)
print(newList.getInfo())

# Extracting value from a list

value = myList.get(0)
print(value.getInfo())

# Casting

# Let's try to do some computation on the extracted value
#newValue = value.add(1)
#print(newValue)

# You get an error because Earth Engine doesn't know what is the type of 'value'
# We need to cast it to appropriate type first
value = ee.Number(value)
newValue = value.add(1)
print(newValue.getInfo())

# Dictionary
# Convert javascript objects to EE Objects
data = {'city': 'Bengaluru', 'population': 8400000, 'elevation': 930}
eeData = ee.Dictionary(data)
# Once converted, you can use the methods from the
# ee.Dictionary module
print(eeData.get('city').getInfo())

# Dates
# For any date computation, you should use ee.Date module

date = ee.Date('2019-01-01')
futureDate = date.advance(1, 'year')
print(futureDate.format('Y-M-d').getInfo())

2
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
1
2
Bengaluru
2020-1-1


In [22]:
import datetime
Map = geemap.Map()

s2 = ee.ImageCollection("COPERNICUS/S2")
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241])

now = ee.Date(datetime.datetime.now().timestamp()*1000)
now = ee.Date(now)
past = now.advance(-1, 'month')
# Apply another filter to the collection below to filter images
# collected in the last 1-month
# Do not hard-code the dates, it should always show images
# from the past 1-month whenever you run the script
# Hint: Use ee.Date.advance() function
#   to compute the date 1 month before now
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(geometry))  \
  .filter(ee.Filter.date(past, now))

Map.centerObject(geometry, 10)

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

mean = filtered.mean()
Map.addLayer(mean, rgbVis, 'last month mean')
Map

Map(center=[12.952912912328241, 77.60412933051538], controls=(WidgetControl(options=['position', 'transparent_…

In [46]:
Map = geemap.Map()

s2 = ee.ImageCollection("COPERNICUS/S2")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

bangalore = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))
geometry = bangalore.geometry()

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image = filtered.median()

# Calculate  Normalized Difference Vegetation Index (NDVI)
# 'NIR' (B8) and 'RED' (B4)
ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])

# Calculate Modified Normalized Difference Water Index (MNDWI)
# 'GREEN' (B3) and 'SWIR1' (B11)
mndwi = image.normalizedDifference(['B3', 'B11']).rename(['mndwi'])

# Calculate Soil-adjusted Vegetation Index (SAVI)
# 1.5 * ((NIR - RED) / (NIR + RED + 0.5))

# For more complex indices, you can use the expression() function

# Note:
# For the SAVI formula, the pixel values need to converted to reflectances
# Multiplyng the pixel values by 'scale' gives us the reflectance value
# The scale value is 0.0001 for Sentinel-2 dataset

savi = image.expression(
    '1.5 * ((NIR - RED) / (NIR + RED + 0.5))', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001),
}).rename('savi')

rgbVis = {'min': 0.0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}
ndwiVis = {'min':0, 'max':0.5, 'palette': ['white', 'blue']}

Map.addLayer(image.clip(geometry), rgbVis, 'Image')
Map.addLayer(mndwi.clip(geometry), ndwiVis, 'mndwi')
Map.addLayer(savi.clip(geometry), ndviVis, 'savi')
Map.addLayer(ndvi.clip(geometry), ndviVis, 'ndvi')
Map.centerObject(geometry, 10)
Map

Map(center=[12.942208526432207, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…

In [53]:
Map = geemap.Map()

s2 = ee.ImageCollection("COPERNICUS/S2")
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

bangalore = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Bangalore Urban'))
geometry = bangalore.geometry()
Map.centerObject(geometry)

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
  .filter(ee.Filter.bounds(geometry))

image = filtered.median()

# Exercise

# Calculate the Normalized Difference Built-Up Index (NDBI) for the image
# Hint: NDBI = (SWIR1 – NIR) / (SWIR1 + NIR)
# Visualize the built-up area using a 'red' palette

ndbi = image.expression('(SWIR1 - NIR) / (SWIR1 + NIR)', {
    'SWIR1': image.select('B11'),
    'NIR'  : image.select('B8')}).rename(['NDBI'])


ndbiVis = {'min':0, 'max':0.5, 'palette': ['white', 'red']}

Map.addLayer(ndbi.clip(geometry), ndbiVis, 'ndbi')

Map.centerObject(geometry, 10)

Map

Map(center=[12.942208526432207, 77.5868323785926], controls=(WidgetControl(options=['position', 'transparent_b…